# WebScraping

In [1]:
from dogpile.cache import make_region
import requests
from lxml import html
import csv
import pandas as pd
import pickle

In [2]:
from dogpile.cache import make_region

cache = make_region().configure(
    'dogpile.cache.memory',
    expiration_time = 24*2600
)

In [3]:
# Visualisation utils from Kris Joanidis
import IPython.display
def display_html(string, script = False, iframe = False, style  = False):
    '''Display HTML string in iPython notebook. <script> tags are removed by default.'''
    root = html.fromstring(string) 
    if not script:
        for element in root.iter("script"):
            element.drop_tree()
    if not iframe:
        for element in root.iter("iframe"):
            element.drop_tree()
    if not style:
        for element in root.iter("style"):
            element.drop_tree()
        for element in root.iter("link"):
            if element.attrib['rel'] == "stylesheet":
                element.drop_tree()
    IPython.display.display(IPython.display.HTML(html.tostring(root).decode('utf-8')))

## WebScrape Box office mojo by year
http://www.boxofficemojo.com/yearly/chart/?page="+str(pg)+"&view=releasedate&view1=domestic&yr="+str(yr)+"&p=.htm"


In [6]:
# @cache.cache_on_arguments()
def pull_page(yr):
    url = "http://www.boxofficemojo.com/oscar/chart/?yr="+str(yr)+"&view=allmovies&p=.htm"
    data = requests.get(url).text
    return data

def pull_mpage(yr,pg):
    url = "http://www.boxofficemojo.com/yearly/chart/?page="+str(pg)+"&view=releasedate&view1=domestic&yr="+str(yr)+"&p=.htm"
    data = requests.get(url).text
    return data
    
def make_soup(data):
    soup = BeautifulSoup(data,"lxml")
    return soup

In [7]:
## 2002 is the first year they add 'close' column to table. 

def mojo_movies():
    nmdf_list = []
    for yr in range(2002,2016):
        pg = 1
        page = html.fromstring(pull_mpage(yr,pg))
        tables = page.cssselect('table table table')
        while len(tables) > 0 :
            df = pd.read_html(html.tostring(tables[1]),header=0)[0]
            df.columns = ["Rank","Title","Studio", "Total_Gross","Gross_no_Theaters", "Opening",
                          "Opening_no_Theaters","Open_date","Close_date"]
            df = df.ix[df.index[1:-3]]

            df['Year'] = yr
            nmdf_list.append(df)

            pg += 1
            page = html.fromstring(pull_mpage(yr,pg))
            tables = page.cssselect('table table table')

    return nmdf_list

In [9]:
## PIPELINE
recent = mojo_movies()
df1 = pd.concat(recent)
df1.to_pickle('2002to2016movie_data.pkl')

## WebScrape Box office mojo by oscars
http://www.boxofficemojo.com/oscar/chart/?yr="+str(yr)+"&view=allmovies&p=.htm

In [13]:
def mojo_oscars():
    nmdf_list = []
    for yr in range(1980,2016):
        page = html.fromstring(pull_page(yr))
        tables = page.cssselect('table table')
        print tables
        df = pd.read_html(html.tostring(tables[1]),header=0)[0]
        df.columns = ["Row","YRRank","Picture","Studio","Box Office","Noms","Wins"]
        df['Year'] = yr
        nmdf_list.append(df)
    return nmdf_list

In [16]:
## PIPELINE
# recent = mojo_oscars()
# df1 = pd.concat(recent)
# df1.to_pickle('oscar2.pkl')

In [17]:
df1.head()

,Rank,Title,Studio,Total_Gross,Gross_no_Theaters,Opening,Opening_no_Theaters,Open_date,Close_date,Year
1,1,Spider-Man,Sony,"$403,706,375",3876,"$114,844,116",3615,5/3,8/18,2002
2,2,The Lord of the Rings: The Two Towers,NL,"$339,789,881",3622,"$62,007,528",3622,12/18,8/24,2002
3,3,Star Wars: Episode II - Attack of the Clones,Fox,"$302,191,252",3161,"$80,027,814",3161,5/16,11/3,2002
4,4,Harry Potter and the Chamber of Secrets,WB,"$261,988,482",3682,"$88,357,488",3682,11/15,5/4,2002
5,5,My Big Fat Greek Wedding,IFC,"$241,438,208",2016,"$597,362",108,4/19,4/13,2002


## IMDB WebScrape
1. http://www.imdb.com/find?ref_=nv_sr_fn&q=  (search for movie title)
2. http://www.imdb.com/title/tt0076759/?ref_=fn_al_tt_1 (pull movie from search results)
3. capture ['IMDB_Score','Users','Metascore','Director','Writers','Actors','Genres','Rating','Title']

In [18]:
BASE_URL = 'http://www.imdb.com'

In [19]:
@cache.cache_on_arguments()


def imdb_data(title):
#     movie = '+'.join(title.split())
    title = '"%s"' % title
    movie = '+'.join(title.split())
    url = "%s/find?s=tt&q=%s" % (BASE_URL, movie)
    page = html.fromstring(requests.get(url).text)
    try: 
        movie_list = page.cssselect('table.findList tr td.result_text a')
#     display_html(html.tostring(page.cssselect('table.findList')[0]))
    except:
        movie_list = []
        failed_movies.append(title)
        print "failedHere"
    return movie_list

def imdb_page(movie_list):
#     if len(movie_list) >0:
    movie_link = movie_list[0].get('href')
    movie_page_url = BASE_URL + movie_link
    next_page = html.fromstring(requests.get(movie_page_url).text)
    print "next_page gotten"
    return next_page

def imdb_pull(next_page):
#     next_page = html.fromstring(requests.get(next_page).text)
    imdb_rating = next_page.cssselect('table div[class=star-box-details] span[itemprop=ratingValue]')[0].text_content()
#     print imdb_rating
    genres = next_page.cssselect('table div[class=infobar] span[itemprop="genre"]')
    genre_list = tuple([genre.text_content() for genre in genres])
#     rating = next_page.cssselect('table div[class=infobar] meta.content')[0]
#     print genre_list
    rating = next_page.cssselect('table div[class=infobar] meta')[0].get('content')
#     print rating
    users = next_page.cssselect('table div[class=star-box-details] span[itemprop=ratingCount]')[0].text_content()
#     print users
    metascore = next_page.cssselect('table div[class=star-box-details] a[href="criticreviews?ref_=tt_ov_rt"]')[0].text_content()
#     print metascore
    director = next_page.cssselect('table div[itemprop="director"] span[itemprop=name]')[0].text_content()
#     print director
    writers = next_page.cssselect('table div[itemprop="creator"] span[itemprop=name]')
    writer_list = tuple([writer.text_content() for writer in writers])
#     print writer_list
    actors = next_page.cssselect('table div[itemprop="actors"] span[itemprop=name]')
    actor_list = tuple([actor.text_content() for actor in actors])
#     print actor_list
    return [imdb_rating, users, metascore, director, writer_list, actor_list, genre_list,rating]



In [55]:
n_failed_movies = []

In [58]:
columns=["IMDB_Score","Users","Metascore","Director","Writers","Actors","Genres","Rating"]
imdb_df =pd.DataFrame(columns=columns) 

for title in ['Zoolander','Star Wars']:
    movie_list = imdb_data(title) 
    if len(movie_list) > 0:
        next_page = imdb_page(movie_list)
        try:
            row = imdb_pull(next_page)
            dictionary = dict(zip(columns,row))
            dictionary['Title'] = title
            imdb_df = imdb_df.append(dictionary, ignore_index=True)
        except:
            print "failed"
            n_failed_movies.append(title)
            continue

next_page gotten
failed
next_page gotten
['8.7', '855,336', ' 92/100\n', 'George Lucas', ('George Lucas',), ('Mark Hamill', 'Harrison Ford', 'Carrie Fisher'), ('Action', 'Adventure', 'Fantasy'), 'PG']


In [ ]:
imdb_df.to_pickle('imdb_2')

In [ ]:
# failed_movies
with open('imdb_failed_movies.pkl', 'w') as picklefile:
    pickle.dump(failed_movies, picklefile)

In [27]:
len(n_failed_movies)


10

In [21]:
#unpickle
with open("titlelist.pkl", 'r') as picklefile: 
    movie_titles = pickle.load(picklefile)
with open("imdb_t6.pkl", 'r') as picklefile: 
    completed = pickle.load(picklefile)
with open("imdb_failed_movies.pkl", 'r') as picklefile: 
    failed_movies = pickle.load(picklefile)

In [9]:
m_titles = list(movie_titles.values)
c_titles = list(completed.Title.values)


In [18]:
ic_titles = set(m_titles) - (set(m_titles) & (set().union(*[c_titles,failed_movies])))
ic = sorted(ic_titles, reverse=True)


In [19]:
len(ic)

410

In [20]:
ic = set(ic) - set([u"Le combat dans l'\xeele (1962)",u'A Prophet (Un proph\xe8te)'])
#  u"Le combat dans l'\xeele (1962)"

In [21]:
ic

{'101 Dalmatians (Re-issue) (1985)',
 '101 Dalmatians (Re-issue) (1991)',
 '1995 Bugs Bunny Film Festival',
 '1998 Bugs Bunny Film Festival',
 '2001: A Space Odyssey (2013 re-release)',
 '2001: A Space Odyssey (Re-issue)',
 '8 1/2 (re-issue)',
 'A Christmas Story (Re-issue)',
 "A Hard Day's Night (2014 re-release)",
 "A Hard Day's Night (Re-issue)",
 'A Man Vanishes (2012 re-release)',
 'A Pig Across Paris (2013 re-issue)',
 'A Poem Is a Naked Person (2015 re-issue)',
 'A Trip to the Moon/The Extraordinary Voyage',
 'Aarakshan (Reservation)',
 "After Dark's Horror Fest: 8 Films to Die For",
 "After Dark's Horrorfest 2",
 'After the Life: Trilogy 3',
 'Aftershock (Tangshan Dadizhen)',
 'Aida at La Scala',
 'Akira (Re-issue)',
 'All About Eve (re-issue)',
 'Alphaville (2013 re-release)',
 "Amos Kolleck's Bad Girls",
 'An Amazing Couple: Trilogy 2',
 'Anchorman 2 (R-Rated Extended Cut)',
 'Andrei Tarkovsky Retrospective',
 'Antoine and Antoinette (2013 re-issue)',
 'Asterix et Obelix: Mis

In [22]:
failed_movies

['whaledreamers',
 'Zyzzyx Road',
 'Zus & Zo',
 'Zoot Suit',
 'Zoolander',
 'Zoo',
 'Zombie High',
 'Zokkomon',
 'Zindagi Rocks',
 'Zindagi Na Milegi Dobara',
 'Zinda',
 'Zeus and Roxanne',
 'Zero Patience',
 'Zero Kelvin',
 'Zero Effect',
 'Zentropa',
 'Zenith',
 'Zelig',
 'Zebrahead',
 'Zapped!',
 'Yuvvraaj',
 'Yours, Mine and Ours',
 'Your Mommy Kills Animals',
 'Your Friends and Neighbors',
 'Youngblood',
 'Young and Wild',
 'Young and Beautiful',
 'Young Soul Rebels',
 'Young Sherlock Holmes',
 'Young Guns II',
 'Young Girls of Rochefort (Re-issue)',
 'Young Einstein',
 'Young Doctors in Love',
 'Young Detective Dee: Rise of the Sea Dragon',
 "You're Gonna Miss Me",
 "You'll Get Over It",
 'You Will Be My Son',
 'You So Crazy',
 "You Ain't Seen Nothin' Yet",
 'Yossi',
 'Yor: Hunter from the Future',
 'Yes, Giorgio',
 'Yentl',
 'Yellowbeard',
 'Yellow Day (2015)',
 'Yellow Day',
 'Yellow Asphalt',
 'Yella',
 'Yeh Jawaani Hai Deewani',
 'Year of the Gun',
 'Year of the Dragon',
 'Ye

In [82]:
## http://data.bls.gov/pdq/SurveyOutputServlet 
cpi_excel = open('SeriesReport.xlsx','r')
cpidf = pd.read_excel(cpi_excel,header=9).set_index('Year')
cpi = cpidf.drop({'HALF1','HALF2'},axis=1)

In [93]:
completed.head()

,IMDB_Score,Users,Metascore,Director,Writers,Actors,Genres,Rating,Title
0,7.3,"516,078",73/100\n,Sam Raimi,"(Stan Lee, Steve Ditko)","(Tobey Maguire, Kirsten Dunst, Willem Dafoe)","(Action, Adventure)",PG-13,Spider-Man
1,8.7,"1,037,342",88/100\n,Peter Jackson,"(J.R.R. Tolkien, Fran Walsh)","(Elijah Wood, Ian McKellen, Viggo Mortensen)","(Adventure, Drama, Fantasy)",PG-13,The Lord of the Rings: The Two Towers
2,6.7,"437,662",54/100\n,George Lucas,"(George Lucas, Jonathan Hales)","(Hayden Christensen, Natalie Portman, Ewan McG...","(Action, Adventure, Fantasy)",PG,Star Wars: Episode II - Attack of the Clones
3,7.4,"366,162",63/100\n,Chris Columbus,"(J.K. Rowling, Steve Kloves)","(Daniel Radcliffe, Rupert Grint, Emma Watson)","(Adventure, Family, Fantasy)",PG,Harry Potter and the Chamber of Secrets
4,6.7,"262,736",59/100\n,M. Night Shyamalan,"(M. Night Shyamalan,)","(Mel Gibson, Joaquin Phoenix, Rory Culkin)","(Drama, Sci-Fi, Thriller)",PG-13,Signs


In [106]:
## Base Period:  1982-84=100
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
cpi.head(n=10)
# cpi['Jan'][1980]
# plt.plot()
cpi.keys()pandas.to_datetime(*args, 


Index([u'Jan', u'Feb', u'Mar', u'Apr', u'May', u'Jun', u'Jul', u'Aug', u'Sep',
       u'Oct', u'Nov', u'Dec'],
      dtype='object')

In [124]:
unstacked_cpi = cpi.unstack()

In [127]:
unstacked_cpi.reset_index()

,level_0,Year,0
0,Jan,1980,77.800
1,Jan,1981,87.000
2,Jan,1982,94.300
3,Jan,1983,97.800
4,Jan,1984,101.900
5,Jan,1985,105.500
6,Jan,1986,109.600
7,Jan,1987,111.200
8,Jan,1988,115.700
9,Jan,1989,121.100


In [118]:
unstacked_cpi_df = pd.DataFrame(unstacked_cpi).reset_index()

In [121]:
unstacked_cpi_df.columns = ['month', 'year','cpi']

In [122]:
unstacked_cpi_df.head()

,month,year,cpi
0,Jan,1980,77.8
1,Jan,1981,87.0
2,Jan,1982,94.3
3,Jan,1983,97.8
4,Jan,1984,101.9


In [117]:
unstacked_cpi_df["Date"] = df.bar.map(str) + " is " + df.foo

,level_0,Year,0
0,Jan,1980,77.800
1,Jan,1981,87.000
2,Jan,1982,94.300
3,Jan,1983,97.800
4,Jan,1984,101.900
5,Jan,1985,105.500
6,Jan,1986,109.600
7,Jan,1987,111.200
8,Jan,1988,115.700
9,Jan,1989,121.100
